In [1]:
import json
from subprocess import Popen

import librosa
import torch
import pretty_midi
import audiocraft
from audiocraft.models.encodec import HFEncodecCompressionModel

from shared import *
from music import PIANO_RANGE

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
c:\Users\iGlop\anaconda3\envs\env_neural_avh\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We have CUDA.
gpu_name = 'NVIDIA GeForce RTX 3050 Ti Laptop GPU'


In [2]:
(DENSITY_MU, DENSITY_SIGMA) = (2.520, 0.672)
(DURATION_MU, DURATION_SIGMA) = (-1.754, 1.077)
(VELOCITY_MU, VELOCITY_SIGMA) = (84.174, 25.561)
(PITCH_MU, PITCH_SIGMA) = (60.229, 13.938)

In [3]:
encodec = audiocraft.models.encodec.CompressionModel.get_pretrained(
    'facebook/encodec_32khz', DEVICE, 
)
assert isinstance(encodec, HFEncodecCompressionModel)
encodec.eval()
None

c:\Users\iGlop\anaconda3\envs\env_neural_avh\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
i = 0

In [5]:
midi = pretty_midi.PrettyMIDI()
piano = pretty_midi.Instrument(0, is_drum=False, name='Piano')
midi.instruments.append(piano)
song_len = float(SEC_PER_DATAPOINT)
density = torch.randn((1, )).mul(DENSITY_SIGMA).add(DENSITY_MU).exp().item()
n_notes = round(song_len * density)
onsets = torch.rand((n_notes, )).mul(song_len).numpy()
durations = torch.randn((n_notes, )).mul(DURATION_SIGMA).add(DURATION_MU).exp().numpy()
velocities = torch.randn((n_notes, )).mul(VELOCITY_SIGMA).add(VELOCITY_MU).clamp(1, 127).round().numpy()
pitches = torch.randn((n_notes, )).mul(PITCH_SIGMA).add(PITCH_MU).round().numpy()
pitches[np.logical_or(
    pitches < PIANO_RANGE[0], 
    pitches > PIANO_RANGE[1], 
)] = 60
for onset, duration, velocity, pitch in zip(
    onsets, durations, velocities, pitches, 
):
    piano.notes.append(pretty_midi.Note(
        velocity=int(velocity), pitch=int(pitch), 
        start=onset, end=min(onset + duration, song_len),
    ))


In [6]:
midi_path = path.join(
    TRANSFORMER_PIANO_DATASET_DIR, f'{i}.mid',
)
midi.write(midi_path)
print(midi_path)


c:\Users\iGlop\d\neuralAVH\unsupervised_symbolic_music_decipher\data\transformer_piano_dataset\0.mid


In [7]:
wav_path = path.join(
    TRANSFORMER_PIANO_DATASET_DIR, 'temp.wav',
)
with Popen([
    'fluidsynth', '-ni', SOUNDFONT_FILE, midi_path,
    '-F', wav_path, '-r', str(ENCODEC_SR), 
]) as p:
    p.wait()

In [8]:
y, _ = librosa.load(wav_path, sr=ENCODEC_SR, mono=True)
wave = torch.Tensor(y).to(DEVICE)
with torch.no_grad():
    codes, _ = encodec.encode(wave.unsqueeze(0).unsqueeze(0))
codes.shape

torch.Size([1, 4, 2685])

In [11]:
len(y) / ENCODEC_SR * 50

2684.7999999999997

In [12]:
codes[0, :, 0]

tensor([ 166, 1931, 2019, 1770], device='cuda:0')

In [13]:
codes.dtype

torch.int64

In [21]:
codes.to(torch.int16)[0, :, 0]

tensor([ 166, 1931, 2019, 1770], device='cuda:0', dtype=torch.int16)

In [19]:
2 ** 15 > 2048

True